In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import contractions
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caokh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
import bz2

data_folder = './data/'
data_file = 'quotes-2019.json.bz2'
data_path = data_folder + data_file

f = bz2.open(data_path, "rb")
data=pd.read_json(f, lines=True, nrows=1000)
data

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-10-14-000009,% 9: D: D @? 6 H2J H6 E9@F89E H6 4@F=5 8: G6 3...,None,[],2019-10-14 07:30:00,1,"[[None, 0.5595], [Julio Morales, 0.4405]]",[http://www.ivpressonline.com/news/local/ivc-f...,E
1,2019-04-08-048753,It is immoral. It is harmful. It is hurtful.,President Donald Trump,[Q22686],2019-04-08 16:22:00,44,"[[President Donald Trump, 0.5802], [None, 0.36...",[https://www.mercedsunstar.com/news/business/a...,E
2,2019-07-17-000030,"[ Amber ] loves her son more than anything,",None,[],2019-07-17 22:54:35,1,"[[None, 0.8276], [Amber Portwood, 0.1724]]",[https://www.inquisitr.com/5535969/did-amber-p...,E
3,2019-05-15-053302,It is important for our equine science student...,Sally Johnson,[Q42336656],2019-05-15 18:03:22,1,"[[Sally Johnson, 0.5721], [None, 0.4279]]",[https://www.lanereport.com/113381/2019/05/qua...,E
4,2019-04-20-000011,... an Afrikaans family living in South Africa...,None,[],2019-04-20 22:30:57,1,"[[None, 0.8331], [you long, 0.1669]]",[http://filmthreat.com/reviews/the-harvester/],E
...,...,...,...,...,...,...,...,...,...
995,2019-03-08-058812,"It's kind of what I talked to the team about, ...",Van Horn,[Q21611418],2019-03-08 01:59:48,2,"[[Van Horn, 0.4315], [Lane Burroughs, 0.3121],...",[http://www.arkansasonline.com/news/2019/mar/0...,E
996,2019-04-24-003931,"And with a book called Death Threat, I just fe...",Vivek Shraya,[Q7937601],2019-04-24 20:41:39,1,"[[Vivek Shraya, 0.8852], [None, 0.1148]]",[https://www.straight.com/arts/1232356/verses-...,E
997,2019-10-21-048385,It's less about what they say to each other an...,Nathan Felix,[Q22277761],2019-10-21 00:00:00,1,"[[Nathan Felix, 0.699], [None, 0.1455], [Domin...",[https://www.tpr.org/post/mcnay-hopes-become-n...,E
998,2019-02-02-003282,"And, you allow the challenge flag to be used o...",Reggie Bush,[Q555271],2019-02-02 20:07:21,7,"[[Reggie Bush, 0.9156], [None, 0.0827], [Cardi...",[http://kron4.com/sports/the-big-game/how-regg...,E


In [28]:
from src.prep_utilities import * 
from src.prep_pipeline import *

In [29]:
import timeit

speaker_file = 'speaker_attributes_processed.csv.bz2'
speaker_path = data_folder + speaker_file


start = timeit.default_timer()
speaker_attributes = pd.read_csv(speaker_path, compression='bz2',index_col='id')

# Drop NaN gender rows
speaker_attributes = speaker_attributes.dropna(subset=['gender'])

# Set id as index
#speaker_attributes = speaker_attributes.set_index('id')

stop = timeit.default_timer()
print('Time: ', stop - start)

d:\EPFL\ADA\ada-2021-project-madam\env\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,9,13,18,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Time:  85.86816240000007


In [32]:
speaker_attributes.head()

,Unnamed: 0,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,...,type,religion,nationality_label,gender_label,ethnic_group_label,occupation_label,party_label,academic_degree_label,candidacy_label,religion_label
id,,,,,,,,,,,,,,,,,,,,,
Q23,0,['Washington' 'President Washington' 'G. Washi...,['+1732-02-22T00:00:00Z'],['Q161885' 'Q30'],['Q6581097'],1395141751,NaN,W000178,['Q82955' 'Q189290' 'Q131512' 'Q1734662' 'Q294...,['Q327591'],...,item,['Q682443'],"['Great Britain', 'United States of America']",male,NaN,"['politician', 'military officer', 'farmer', '...",independent politician,NaN,"['1792 United States presidential election', '...",Episcopal Church
Q42,1,['Douglas Noel Adams' 'Douglas Noël Adams' 'Do...,['+1952-03-11T00:00:00Z'],['Q145'],['Q6581097'],1395737157,['Q7994501'],NaN,['Q214917' 'Q28389' 'Q6625963' 'Q4853732' 'Q18...,NaN,...,item,NaN,United Kingdom,male,White British,"['playwright', 'screenwriter', 'novelist', ""ch...",NaN,NaN,NaN,NaN
Q1868,2,['Paul Marie Ghislain Otlet' 'Paul Marie Otlet'],['+1868-08-23T00:00:00Z'],['Q31'],['Q6581097'],1380367296,NaN,NaN,['Q36180' 'Q40348' 'Q182436' 'Q1265807' 'Q2053...,NaN,...,item,NaN,Belgium,male,NaN,"['writer', 'lawyer', 'librarian', 'information...",NaN,NaN,NaN,NaN
Q207,3,['George Walker Bush' 'Bush Jr.' 'Dubya' 'GWB'...,['+1946-07-06T00:00:00Z'],['Q30'],['Q6581097'],1395142029,NaN,NaN,['Q82955' 'Q15982858' 'Q18814623' 'Q1028181' '...,['Q29468'],...,item,['Q329646' 'Q682443' 'Q33203'],United States of America,male,NaN,"['politician', 'motivational speaker', 'autobi...",Republican Party,NaN,"['2000 United States presidential election', '...","['United Methodist Church', 'Episcopal Church'..."
Q297,4,['Velázquez' 'Diego Rodríguez de Silva y Veláz...,['+1599-06-06T00:00:00Z'],['Q29'],['Q6581097'],1391704596,NaN,NaN,['Q1028181'],NaN,...,item,NaN,Spain,male,NaN,painter,NaN,NaN,NaN,NaN


In [33]:
start = timeit.default_timer()

data2 = prep_docs(data, speaker_attributes)

stop = timeit.default_timer()

print('Time: ', stop - start)

Copying...
Simplifying date column...
Tokenizing quotes...


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\caokh/nltk_data'
    - 'd:\\EPFL\\ADA\\ada-2021-project-madam\\env\\nltk_data'
    - 'd:\\EPFL\\ADA\\ada-2021-project-madam\\env\\share\\nltk_data'
    - 'd:\\EPFL\\ADA\\ada-2021-project-madam\\env\\lib\\nltk_data'
    - 'C:\\Users\\caokh\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [9]:
data2.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,phase,tokens,websites,gender
0,2019-04-08-048753,It is immoral. It is harmful. It is hurtful.,President Donald Trump,[Q22686],2019-04,44,E,"[be, immoral, be, harmful, be, hurtful]","[mercedsunstar, sacbee, mynorthwest, lasvegass...",male
1,2019-05-15-053302,It is important for our equine science student...,Sally Johnson,[Q42336656],2019-05,1,E,"[be, important, equine, science, student, be, ...",[lanereport],female
2,2019-02-27-055406,It is important to many Native American tribes...,Rafael Ortega,"[Q16672061, Q3417253, Q3417255, Q48410107]",2019-02,1,E,"[be, important, many, native, american, tribe,...",[kstp],male
3,2019-12-08-023053,"It is impossible, biologically, truly to `rest...",Barry Lopez,[Q809063],2019-12,1,E,"[be, impossible, biologically, truly, restore,...",[timescolonist],male
4,2019-02-21-000088,[ Chilton ] put it on a little tape recorder a...,Sam the Sham,[Q1971786],2019-02,1,E,"[chilton, put, little, tape, recorder, take, a...",[nashvillescene],male


In [13]:
df = data2.groupby('gender')['quoteID'].nunique()
df

gender
female                146
male                  584
transgender female      2
Name: quoteID, dtype: int64